In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
train = pd.read_csv('../dataset/train_modified.csv')
valid = pd.read_csv('../dataset/val_modified.csv')
test = pd.read_csv('../dataset/test_modified.csv')

In [4]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 168

In [5]:
xtrain, ytrain = train.Comment.values, train.Emotion.values
xvalid, yvalid = valid.Comment.values, valid.Emotion.values
xtest, ytest = test.Comment.values, test.Emotion.values
                                                  

train['Comment'].apply(lambda x:len(str(x).split())).max()

119

In [6]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 168

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [7]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [8]:
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 168, 300)          2860500   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               40100     
_________________________________________________________________
dense (Dense)                (None, 7)                 707       
Total params: 2,901,307
Trainable params: 2,901,307
Non-trainable params: 0
_________________________________________________________________


In [99]:
model.fit(xtrain_pad, ytrain,epochs=10,validation_data=(xvalid_pad, yvalid), batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/7
9/9 [==============================] - 4s 459ms/step - loss: 1.9428 - accuracy: 0.1652
Epoch 2/7
9/9 [==============================] - 0s 20ms/step - loss: 1.9060 - accuracy: 0.4058
Epoch 3/7
9/9 [==============================] - 0s 20ms/step - loss: 1.8392 - accuracy: 0.5973
Epoch 4/7
9/9 [==============================] - 0s 21ms/step - loss: 1.7280 - accuracy: 0.6880
Epoch 5/7
9/9 [==============================] - 0s 20ms/step - loss: 1.5896 - accuracy: 0.7967
Epoch 6/7
9/9 [==============================] - 0s 20ms/step - loss: 1.4690 - accuracy: 0.8558
Epoch 7/7
9/9 [==============================] - 0s 20ms/step - loss: 1.3895 - accuracy: 0.8857


In [108]:
model.evaluate(xtest_pad, ytest)

33/33 [==============================] - 1s 37ms/step - loss: 1.8760 - accuracy: 0.2777


[1.8759645223617554, 0.2777242064476013]